In [66]:
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
        self.feature = feature # milyen tulajdonság szerint bontunk
        self.threshold = threshold  # milyen küszöbértékkel bontunk
        self.left = left # bal oldali gyerek
        self.right = right # jobb oldali gyerek
        self.value = value # érték, ha levélcsomópont
        
    def is_leaf_node(self):
        #ha van értéke, akkor levélcsomópont
        return self.value is not None
    
    def disp(self):
        print(f"\n\nNode:\nfeature",self.feature, "\t\ntreshold", self.threshold,"\t\nvalue", self.value, "\nleft", self.left, "\nright", self.right, sep='\t')
        if self.left is not None:
            print("LEFT side")
            self.left.disp()
        if self.right is not None:
            print("RIGHT side")
            self.right.disp()

class DecisionTree:
    #max_depth: REQUIRED - maximális mélység
    #min_samples_split: REQUIRED - minimális mintaszám, hogy egy csomópontot szétválasszunk
    #min_samples_leaf: REQUIRED - minimális mintaszám, hogy egy csomópont levél legyen
    def __init__(self, max_depth=10, min_samples_split=2, min_samples_leaf = 1, number_of_features_used=None, random_state=None, split_criterion="information_gain"):
        self.min_samples_split=min_samples_split # minimális mintaszám, hogy egy csomópontot szétválasszunk
        self.max_depth=max_depth # fa maximális mélysége
        self.number_of_features_used=number_of_features_used # mennyi attribútumot használunk a legjobb bontáshoz, attribútumok egy részhalmaza
        self.min_samples_leaf=min_samples_leaf # minimális mintaszám, hogy egy csomópont levél legyen
        self.random_state=random_state # random szám generálásához használt seed, reprodukálhatóság miatt
        self.split_criterion = split_criterion # bontási kritérium, mse vagy information_gain
        self.root=None

    #Modeel tanítása
    def fit(self, X, y):
        # Hiányzó értékek pótlása
        mean = np.nanmean(X, axis=0)  # Oszlopok átlaga (kivéve a hiányzó értékeket)
        indexs = np.where(np.isnan(X))  # Hiányzó értékek helyei
        X[indexs] = np.take(mean, indexs[1])

        self.number_of_features_used = X.shape[1] if not self.number_of_features_used else min(X.shape[1],self.number_of_features_used) # ha paraméterként nem adtuk meg semmit, akkor az összes attribútumot használjuk, egyébként a 2 minimumát
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        #rekurzív bontás a legjobb bontással
        number_of_samples, number_of_features = X.shape #mintaszám, attribútumok száma
        number_of_labels = len(np.unique(y)) #ha ez egy akkor már nem bontjuk tovább -> levélcsomópont
        
        # 1.) Leállási feltételek
        if (depth>=self.max_depth 
            or number_of_labels==1 
            or number_of_samples<self.min_samples_split
            or number_of_samples < self.min_samples_leaf):
            # ha bármelyik feltétel teljesül, akkor levélcsomópontnál tartunk
            # a levélcsomópont értéke a leggyakoribb osztály lesz
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)
        
        #random_seed beállítása, ha meg lett adva, reprodukálhatóság miatt
        if self.random_state is not None:
            np.random.seed(self.random_state) #random seed beállítása

        features_randomly_chosen = np.random.choice(number_of_features, self.number_of_features_used, replace=False)
        #print("features_randomly_chosen: ", features_randomly_chosen)

        # 2.) Legjobb bontás keresése
        best_feature, best_threshold = self._best_split(X, y, features_randomly_chosen) #visszatér a legjobb attribútummal és küszöbértékkel
        #print(best_feature)
        # 3.) Gyermekek létrehozása és rekurzív hívás
        left_indexs, right_indexs = self._split(X[:, best_feature], best_threshold)
        left = self._grow_tree(X[left_indexs, :], y[left_indexs], depth+1)
        right = self._grow_tree(X[right_indexs, :], y[right_indexs], depth+1)
        return Node(feature=best_feature, threshold=best_threshold, left=left, right=right)
    
    #A leggyakoribb érték meghatározása
    def _most_common_label(self, y):
        if len(y) == 0:
            return None

        counter = Counter(map(tuple, y))  # Convert elements to tuples
        #print("\n\n\nCounter: ", counter)
        most_common = counter.most_common(1)
        #print("Counter.most_common(1): ", most_common)

        if len(most_common) == 0:
            return None

        value = most_common[0][0]  # leggyakoribb érték
        return value
    
    #legjobb vágás keresése
    def _best_split(self, X, y, features_randomly_chosen):
        best_criterion = None
        split_index, split_threshold = None, None

        for feature_index in features_randomly_chosen:  #végigmenünk az összes kiválasztott elemen
            X_column = X[:, feature_index]#végigpróbáljuk az összes attribútumot 0->random feature index
            thresholds = np.unique(X_column)
            #print("Thresholds: ", thresholds)

            for threshold in thresholds:
                # minden egyes küszöbértéknél kiszámoljuk a bontás kritériumát
                criterion = self._split_criterion(y, X_column, threshold)

                if best_criterion is None or criterion > best_criterion: #ha nincs még legjobb kritérium vagy a mostani jobb, akkor frissítjük
                    best_criterion = criterion
                    split_index = feature_index
                    split_threshold = threshold

        return split_index, split_threshold

    #lehetséges vágási kritériumok használata
    def _split_criterion(self, y, X_column, threshold):
        if self.split_criterion == "information_gain":
            return self._information_gain(y, X_column, threshold)
        elif self.split_criterion == "mse":
            return self._mean_squared_error(y, X_column, threshold)
        else:
            raise ValueError("Invalid split criterion!")

    #MSE kritérium számítása - TODO: átnézni
    def _mean_squared_error(self, y, X_column, threshold):
        left_idxs, right_idxs = self._split(X_column, threshold)
        left_values = y[left_idxs]
        right_values = y[right_idxs]
        left_mean = np.mean(left_values)
        right_mean = np.mean(right_values)
        mse = np.mean((left_values - left_mean) ** 2) + np.mean((right_values - right_mean) ** 2)
        return mse
    
    #TODO: átnézni
    def _information_gain(self, y, X_column, threshold):
        # parent entropy
        parent_entropy = self._entropy(y)

        # create children
        left_indexs, right_indexs = self._split(X_column, threshold)

        if len(left_indexs) == 0 or len(right_indexs) == 0:
            return 0
        
        # calculate the weighted avgerage entropy of children
        number_of_samples = len(y)
        number_of_samples_left, number_of_samples_right = len(left_indexs), len(right_indexs)
        entropy_left, entorpy_right = self._entropy(y[left_indexs]), self._entropy(y[right_indexs])
        child_entropy = (number_of_samples_left/number_of_samples) * entropy_left + (number_of_samples_right/number_of_samples) * entorpy_right

        # calculate the information gain
        information_gain = parent_entropy - child_entropy
        return information_gain

    #TODO: átnézni
    def _entropy(self, y):
        flattened_y = np.array(y).flatten()  # Flatten the nested object if needed
        histogram = np.bincount(flattened_y.astype(int))  # Compute histogram
        ps = histogram / len(flattened_y)
        return -np.sum([p * np.log(p) for p in ps if p > 0])

    #TODO: átnézni
    def _split(self, X_column, split_threshold):
        left_idxs = np.argwhere(X_column <= split_threshold).flatten()
        right_idxs = np.argwhere(X_column > split_threshold).flatten()
        return left_idxs, right_idxs
    
    #For predicting the output - TODO: átnézni
    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value

        if node.feature is not None and node.threshold is not None:
            if x[node.feature] <= node.threshold:
                return self._traverse_tree(x, node.left)
            else:
                return self._traverse_tree(x, node.right)

        return None  # or a default value based on your requirements


    
    #Fa kiíratása
    def print(self):
        self.root.disp()
        pass

    #Fa másfajta megjeeelenítése
    def print_tree(self):
        self._print_node(self.root)

    #TODO: átnézni
    def _print_node(self, node, indent=''):
        if node is None:
            return

        if node.is_leaf_node():
            print(indent + f"Leaf: {node.value}")
        else:
            print(indent + f"Feature {node.feature} <= {node.threshold} ?")
            print(indent + "--> True:")
            self._print_node(node.left, indent + "    ")
            print(indent + "--> False:")
            self._print_node(node.right, indent + "    ")

    def plot_tree_the_second(self, node, depth, x_offset, y_offset, x_scale):
        if node.is_leaf_node():
            value = node.value
            plt.text(x_offset, y_offset, f'Value: {value:.2f}', fontsize=12, ha='center', va='center',
                    bbox=dict(facecolor='white', edgecolor='black', boxstyle='round'))
        else:
            feature = node.feature
            threshold = node.threshold
            left_child = node.left
            right_child = node.right

            plt.text(x_offset, y_offset, f'Feature {feature}\n<= {threshold:.2f}', fontsize=12, ha='center',
                    va='center', bbox=dict(facecolor='white', edgecolor='black', boxstyle='round'))

            # Kiszámoljuk a következő csomópontok helyét
            next_y_offset = y_offset - 1
            next_x_offset_left = x_offset - x_scale / (2 ** (depth + 1))
            next_x_offset_right = x_offset + x_scale / (2 ** (depth + 1))

            plt.plot([x_offset, next_x_offset_left], [y_offset, next_y_offset], 'k-')
            plt.plot([x_offset, next_x_offset_right], [y_offset, next_y_offset], 'k-')

            self.plot_tree_the_second(left_child, depth + 1, next_x_offset_left, next_y_offset, x_scale)
            self.plot_tree_the_second(right_child, depth + 1, next_x_offset_right, next_y_offset, x_scale)


In [67]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import tree
import matplotlib.pyplot as plt
import pandas as pd

#Minta adatforrás
df = pd.read_csv("winequality-red.csv")
print(df.shape)
display(df.head())
X = df[df.columns[:-1]].values
y = df[df.columns[:-1]].values

#display("X:",X)
#display("y:",y)

"""wine = datasets.load_breast_cancer()
#display(wine)
X = wine.data # adatok
y = wine.target # címkék/célváltozók"""


# Előkészítsük a bemeneti adatokat és a címkéket
#X = np.array([[1, 2, 3], [4, np.nan, 6], [7, 8, 9]])# Példa bemeneti adatok (a második adat hiányzó értéket tartalmaz)
#y = np.array([0, 1, 0]) # Példa címkék



#Adatok felosztása tanító és tesztelő halmazra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0) # 80% tanító, 20% tesztelő
"""display("X_train:",X_train, "X_train.shape:",X_train.shape)
display("y_train:",y_train, "y_train.shape:",y_train.shape)

display("X_test:",X_test, "X_test.shape:",X_test.shape)
display("y_test:",y_test, "y_test.shape:",y_test.shape)"""

#Döntési fa tanítása
decision_tree = DecisionTree(max_depth=10, split_criterion="information_gain", random_state=0)
decision_tree.fit(X_train, y_train)

#Döntési fa kiértékelése
y_pred = decision_tree.predict(X_test)

def accuracy(y_test, y_pred):
    return np.sum(y_test == y_pred) / len(y_test)

acc = accuracy(y_test, y_pred)
print("Accuracy: ", acc)

decision_tree.print_tree()
decision_tree.plot_tree_the_second(decision_tree.root, 0, 0, 0, 1)

#dt.print()

(1599, 12)


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (320,) + inhomogeneous part.